In [ ]:
# needed pkgs
# !pip install pandas
# !pip install sqlite3
# !pip install lanchain
# !pip install lanchain_experimental
import sqlite3


db_name = 'records.db'
table_name = 'financial_records'
csv_file = 'expenses.CSV'

# Connect to database
conn = sqlite3.connect('records.db')
cursor = conn.cursor()

# Drop table to rest 
drop_table_query = f'''DROP TABLE IF EXISTS  {table_name}'''
cursor.execute(drop_table_query)
#Create table
create_table_query = f'''
    CREATE TABLE IF NOT EXISTS {table_name} (
        transaction_date TEXT,
        post_date TEXT,
        description TEXT,
        category TEXT,
        type TEXT,
        amount INTEGER,
        memo TEXT
    );
'''
cursor.execute(create_table_query)

In [ ]:
import pandas as pd

#Read csv
df = pd.read_csv(csv_file,encoding='ISO-8859-1')
df.columns =['transaction_date', 'post_date', 'description', 'category',
       'type', 'amount', 'memo']

#Import the csv into database
df.to_sql(table_name, conn, if_exists='append', index=False)
conn.close()

In [ ]:
# verify connection
conn = sqlite3.connect('records.db')
cursor = conn.cursor()
# cursor.execute('SELECT * FROM financial_records LIMIT 1')
cursor.execute('SELECT transaction_date, COUNT(*) AS count FROM financial_records GROUP BY transaction_date ORDER BY count DESC LIMIT 1')
cursor.fetchall()
conn.close()

In [ ]:
import os 
api_key = os.environ["OPENAI_API_KEY"]

from langchain.llms import OpenAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
db = SQLDatabase.from_uri(f"sqlite:///{db_name}")
llm = OpenAI(temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [ ]:
# db_chain.run('what is the most common value for transaction_date in the financial_records table?')
# db_chain.run('What was the lowest amount? Please return the entire matching row')
# db_chain.run('What was the highest amount? Please return the entire matching row')
# db_chain.run('What was the most common description and total count?')
# db_chain.run('what was the most common category and total count?')
# total expenses
# db_chain.run('please add all the amount value together and multiply that total by -1')
db_chain.run("What category has the lowest amount total?")
# SELECT category, SUM(amount) AS total_amount FROM financial_records GROUP BY category ORDER BY total_amount ASC LIMIT 1
# db_chain.run("What category has the higest amount total?")
# SELECT category, SUM(amount) AS total_amount FROM financial_records GROUP BY category ORDER BY total_amount DESC LIMIT 1